In [2]:
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline, get_baseline_tdx,convert_daily_return_to_pyfolio_ts

In [3]:
from lutils.stock import LTdxHq

In [4]:
ddf = pd.read_pickle('d:/ddf.pkl').dropna()

In [5]:
ddf = ddf.sort_values(['date','tic'], ignore_index=True)
ddf.index = ddf.date.factorize()[0]

In [6]:
dl = ddf.loc[-252:,:]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
corr = dd.cov().corr()
# corr.style.background_gradient(cmap='coolwarm')

# c1 = corr.abs().unstack().sort_values(ascending = True)
c1 = corr.unstack().sort_values(ascending = True)

In [7]:
# corr.style.background_gradient(cmap='coolwarm')

In [8]:
corr.sum().sort_values()

tic
000921   -361.405100
600365   -340.943427
603789   -327.968372
300406   -323.993151
002032   -318.416261
             ...    
002866    814.262574
603528    817.303297
603095    822.564493
002849    822.916285
300780    836.507501
Length: 2376, dtype: float64

In [9]:
stock_codes = list(corr.sum().sort_values().index.values[:6])

In [10]:
ltdxhq = LTdxHq()

indexs = None
dfs = []
for code in stock_codes:
    df = ltdxhq.get_k_data_daily(code, start='2018-01-01') # 2014-01-01
    
    if indexs is None:
        indexs = df.index
    else:
        indexs = indexs.union(df.index)
    
#     df = df.assign(date = df.index)
#     df = df.assign(day = df.index.weekday)
#     df.date = df.date.dt.strftime('%Y-%m-%d')
    df = df.assign(tic = code)
#     df.index = range(df.shape[0])
    
    dfs.append(df)
    print('----------- over %s min: %s max: %s -----------' % (code, df.index.min(), df.index.max()))

for i, df in enumerate(dfs):
    df = df.reindex(indexs)
    df = df.assign(date = df.index)
    df = df.assign(day = df.index.weekday)
    df.index = range(df.shape[0])
    
    dfs[i] = df.ffill()
    
df = pd.concat(dfs)
# df.index = range(df.shape[0])

ltdxhq.close()

----------- over 000921 min: 2018-01-02 00:00:00 max: 2022-01-17 00:00:00 -----------
----------- over 600365 min: 2018-03-20 00:00:00 max: 2022-01-17 00:00:00 -----------
----------- over 603789 min: 2018-01-02 00:00:00 max: 2022-01-17 00:00:00 -----------
----------- over 300406 min: 2018-01-02 00:00:00 max: 2022-01-17 00:00:00 -----------
----------- over 002032 min: 2018-01-02 00:00:00 max: 2022-01-17 00:00:00 -----------
----------- over 601868 min: 2021-09-28 00:00:00 max: 2022-01-17 00:00:00 -----------


In [11]:
fe = FeatureEngineer(use_technical_indicator=True,
                     use_turbulence=False,
                     user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [12]:
# add covariance matrix as states
df=df.sort_values(['date','tic'], ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback, len(df.index.unique())):
    data_lookback = df.loc[i-lookback:i,:]
    price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list': cov_list,'return_list': return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [13]:
df['daily_variance'] = (df.high-df.low) / df.close

In [14]:
dl = df.loc[-252:, :]
dd = dl.pivot_table(index = 'date',columns = 'tic', values = 'close').pct_change().dropna()
dd.cov().corr().style.background_gradient(cmap='coolwarm')

tic,000921,002032,300406,603789
tic,,,,
000921,1.000000,-0.003048,-0.083801,-0.495833
002032,-0.003048,1.000000,-0.363685,-0.609487
300406,-0.083801,-0.363685,1.000000,-0.308539
603789,-0.495833,-0.609487,-0.308539,1.000000


In [41]:
indexes = df.date.unique()

In [20]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                 df,
                 stock_dim,
                 hmax,
                 initial_amount,
                 transaction_cost_pct,
                 reward_scaling,
                 state_space,
                 action_space,
                 tech_indicator_list,
                 turbulence_threshold=None,
                 lookback=252,
                 day=0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low=0, high=1,shape=(self.action_space,)) 
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1 / self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique()) - 1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
#             plt.plot(df.daily_return.cumsum(),'r')
#             plt.savefig('results/cumulative_reward.png')
#             plt.close()
            
#             plt.plot(self.portfolio_return_memory,'r')
#             plt.savefig('results/rewards.png')
#             plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() != 0:
                sharpe = (252 ** 0.5) * df_daily_return['daily_return'].mean() / df_daily_return['daily_return'].std()
                print("Sharpe: ", sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values) - 1) * weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values) * weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator / denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

Stock Dimension: 4, State Space: 4


{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cuda device


begin_total_asset:100000
end_total_asset:182465.48026857513
Sharpe:  1.1065896459201645
begin_total_asset:100000
end_total_asset:178809.73289876245
Sharpe:  1.0807434878379643
begin_total_asset:100000
end_total_asset:181121.24013149634
Sharpe:  1.1058829515338124
----------------------------------
| time/              |           |
|    fps             | 487       |
|    iterations      | 1         |
|    time_elapsed    | 4         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 143338.14 |
----------------------------------
begin_total_asset:100000
end_total_asset:148602.91890077994
Sharpe:  0.7858707973754905
begin_total_asset:100000
end_total_asset:162595.19967599778
Sharpe:  0.9314661439012591
begin_total_asset:100000
end_total_asset:168517.819030925
Sharpe:  0.9937940825653016
------------------------------------------
| time/                   |              |
|    fps                  | 482          |
|    iterations           | 2 

In [ ]:
initial_amount = 100000
for date in indexes[-126:]:
    train = data_split(df, '2019-01-01', date)
    
    stock_dimension = len(train.tic.unique())
    state_space = stock_dimension
    
    
    tech_indicator_list = ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'daily_variance', 'dx_30', 'close_30_sma', 'close_60_sma']
    env_kwargs = {
        "hmax": 100, 
        "initial_amount": initial_amount, 
        "transaction_cost_pct": 0, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": tech_indicator_list, # config.TECHNICAL_INDICATORS_LIST, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-1

    }

    e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
    env_train, _ = e_train_gym.get_sb_env()
    
    agent = DRLAgent(env = env_train)
    PPO_PARAMS = {
        "n_steps": 2048,
        "ent_coef": 0.005,
        "learning_rate": 0.0001,
        "batch_size": 128,
    }
    model_ppo = agent.get_model("ppo", model_kwargs = PPO_PARAMS)
    
    trained_ppo = agent.train_model(model=model_ppo, tb_log_name='ppo', total_timesteps=40000)
    
    ################################################
    trade = data_split(df, date, '2021-10-22')
    
    e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)
    
    df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=trained_ppo, environment=e_trade_gym)
    time_ind = pd.Series(df_daily_return_ppo.date)
    ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
    DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)
    
    initial_amount = initial_amount * (ppo_cumpod[1] + 1)


In [48]:
perf_stats_all_ppo

Annual return           1.352837
Cumulative returns      0.006814
Annual volatility       0.076484
Sharpe ratio           11.224972
Calmar ratio                 NaN
Stability               1.000000
Max drawdown            0.000000
Omega ratio                  NaN
Sortino ratio                inf
Skew                    0.000000
Kurtosis               -2.000000
Tail ratio             19.000000
Daily value at risk    -0.006229
Alpha                   0.000000
Beta                    1.000000
dtype: float64

In [50]:
df_daily_return_ppo

,date,daily_return
0,2021-10-20,0.000000
1,2021-10-21,0.013957


In [55]:
ppo_cumpod[1]

0.013956783070495193

In [52]:
DRL_strat_ppo

date
2021-10-20 00:00:00+00:00    0.000000
2021-10-21 00:00:00+00:00    0.013957
dtype: float64

In [49]:
from pyfolio import timeseries

df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=trained_ppo, environment=e_trade_gym)
time_ind = pd.Series(df_daily_return_ppo.date)
ppo_cumpod =(df_daily_return_ppo.daily_return + 1).cumprod() - 1
DRL_strat_ppo = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)

# perf_func = timeseries.perf_stats 
# perf_stats_all_ppo = perf_func(returns=DRL_strat_ppo, 
#                                factor_returns=DRL_strat_ppo, 
#                                positions=None, transactions=None, turnover_denom="AGB")

begin_total_asset:100000
end_total_asset:101395.67830704951
Sharpe:  11.224972160321824
hit end!
